<a href="https://colab.research.google.com/github/Gayaz2000/vllm_test/blob/main/vLLM_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vllm --upgrade

#Offline Batched Inference

- LLM is the main class for running offline inference with vLLM engine.
- SamplingParams specifies the parameters for the sampling process.

# Offline Inference

## Automatic Prefix Caching  (Encountering OOM on T4)

In [1]:
import time
from vllm import LLM, SamplingParams

LONG_PROMPT = (
    "You are a helpful assistant in recognizes the content of tables in markdown format. Here is a table as follows.\n# Table\n"
    + """
| ID  | Name          | Age | Occupation    | Country       | Email                  | Phone Number   | Address                       |
|-----|---------------|-----|---------------|---------------|------------------------|----------------|------------------------------|
| 1   | John Doe      | 29  | Engineer      | USA           | john.doe@example.com   | 555-1234       | 123 Elm St, Springfield, IL  |
| 2   | Jane Smith    | 34  | Doctor        | Canada        | jane.smith@example.com | 555-5678       | 456 Oak St, Toronto, ON      |
| 3   | Alice Johnson | 27  | Teacher       | UK            | alice.j@example.com    | 555-8765       | 789 Pine St, London, UK      |
| 4   | Bob Brown     | 45  | Artist        | Australia     | bob.b@example.com      | 555-4321       | 321 Maple St, Sydney, NSW    |
| 5   | Carol White   | 31  | Scientist     | New Zealand   | carol.w@example.com    | 555-6789       | 654 Birch St, Wellington, NZ |
| 6   | Dave Green    | 28  | Lawyer        | Ireland       | dave.g@example.com     | 555-3456       | 987 Cedar St, Dublin, IE     |
| 7   | Emma Black    | 40  | Musician      | USA           | emma.b@example.com     | 555-1111       | 246 Ash St, New York, NY     |
| 8   | Frank Blue    | 37  | Chef          | Canada        | frank.b@example.com    | 555-2222       | 135 Spruce St, Vancouver, BC |
| 9   | Grace Yellow  | 50  | Engineer      | UK            | grace.y@example.com    | 555-3333       | 864 Fir St, Manchester, UK   |
| 10  | Henry Violet  | 32  | Artist        | Australia     | henry.v@example.com    | 555-4444       | 753 Willow St, Melbourne, VIC|
| 11  | Irene Orange  | 26  | Scientist     | New Zealand   | irene.o@example.com    | 555-5555       | 912 Poplar St, Auckland, NZ  |
| 12  | Jack Indigo   | 38  | Teacher       | Ireland       | jack.i@example.com     | 555-6666       | 159 Elm St, Cork, IE         |
| 13  | Karen Red     | 41  | Lawyer        | USA           | karen.r@example.com    | 555-7777       | 357 Cedar St, Boston, MA     |
| 14  | Leo Brown     | 30  | Chef          | Canada        | leo.b@example.com      | 555-8888       | 246 Oak St, Calgary, AB      |
| 15  | Mia Green     | 33  | Musician      | UK            | mia.g@example.com      | 555-9999       | 975 Pine St, Edinburgh, UK   |
| 16  | Noah Yellow   | 29  | Doctor        | Australia     | noah.y@example.com     | 555-0000       | 864 Birch St, Brisbane, QLD  |
| 17  | Olivia Blue   | 35  | Engineer      | New Zealand   | olivia.b@example.com   | 555-1212       | 753 Maple St, Hamilton, NZ   |
| 18  | Peter Black   | 42  | Artist        | Ireland       | peter.b@example.com    | 555-3434       | 912 Fir St, Limerick, IE     |
| 19  | Quinn White   | 28  | Scientist     | USA           | quinn.w@example.com    | 555-5656       | 159 Willow St, Seattle, WA   |
| 20  | Rachel Red    | 31  | Teacher       | Canada        | rachel.r@example.com   | 555-7878       | 357 Poplar St, Ottawa, ON    |
| 21  | Steve Green   | 44  | Lawyer        | UK            | steve.g@example.com    | 555-9090       | 753 Elm St, Birmingham, UK   |
| 22  | Tina Blue     | 36  | Musician      | Australia     | tina.b@example.com     | 555-1213       | 864 Cedar St, Perth, WA      |
| 23  | Umar Black    | 39  | Chef          | New Zealand   | umar.b@example.com     | 555-3435       | 975 Spruce St, Christchurch, NZ|
| 24  | Victor Yellow | 43  | Engineer      | Ireland       | victor.y@example.com   | 555-5657       | 246 Willow St, Galway, IE    |
| 25  | Wendy Orange  | 27  | Artist        | USA           | wendy.o@example.com    | 555-7879       | 135 Elm St, Denver, CO       |
| 26  | Xavier Green  | 34  | Scientist     | Canada        | xavier.g@example.com   | 555-9091       | 357 Oak St, Montreal, QC     |
| 27  | Yara Red      | 41  | Teacher       | UK            | yara.r@example.com     | 555-1214       | 975 Pine St, Leeds, UK       |
| 28  | Zack Blue     | 30  | Lawyer        | Australia     | zack.b@example.com     | 555-3436       | 135 Birch St, Adelaide, SA   |
| 29  | Amy White     | 33  | Musician      | New Zealand   | amy.w@example.com      | 555-5658       | 159 Maple St, Wellington, NZ |
| 30  | Ben Black     | 38  | Chef          | Ireland       | ben.b@example.com      | 555-7870       | 246 Fir St, Waterford, IE    |
"""
)

def generation_time(llm, sampling_params, prompts):
  """Time for response generation"""
  start_time = time.time()
  output = llm.generate(prompts, sampling_params)
  process_time = time.time() - start_time
  print("-" * 30)
  print(f"Output: {output[0].outputs[0].text}")
  print(f"Generation time: {process_time} seconds.")
  print("-" * 30)


INFO 07-23 09:02:10 [__init__.py:244] Automatically detected platform cuda.


In [2]:
def main():
  try:
      llm = LLM(model="meta-llama/Llama-3.2-1B", enable_prefix_caching=True)

      sampling_params = SamplingParams(temperature=0, max_tokens=100)
      generation_time(
          llm,
          sampling_params,
          LONG_PROMPT
          + "Question: what is the age of John Doe? Your answer: The age of John Doe is ",
      )

      generation_time(
          llm,
          sampling_params,
          LONG_PROMPT
          + "Question: what is the age of Zack Blue? Your answer: The age of Zack Blue is ",
      )

  except Exception as e:
    print(e)


if __name__ == "__main__":
    main()

INFO 07-23 09:02:27 [config.py:841] This model supports multiple tasks: {'embed', 'classify', 'reward', 'generate'}. Defaulting to 'generate'.
WARNING 07-23 09:02:27 [config.py:3320] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 07-23 09:02:27 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 07-23 09:02:27 [config.py:1472] Using max model len 131072
WARNING 07-23 09:02:27 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 07-23 09:02:27 [arg_utils.py:1531] Chunked prefill is enabled by default for models with max_model_len > 32K. Chunked prefill might not work with some features or models. If you encounter any issues, please disable by launching with --enable-chunked-prefill=False.
INFO 07-23 09:02:27 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 07-23 09:02:27 [llm_engine.py:230] Ini

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-23 09:02:34 [default_loader.py:272] Loading weights took 3.86 seconds
INFO 07-23 09:02:35 [model_runner.py:1203] Model loading took 2.3185 GiB and 4.453004 seconds
INFO 07-23 09:02:36 [worker.py:294] Memory profiling takes 0.98 seconds
INFO 07-23 09:02:36 [worker.py:294] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 07-23 09:02:36 [worker.py:294] model weights take 2.32GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.18GiB; the rest of the memory reserved for KV Cache is 9.72GiB.
INFO 07-23 09:02:37 [executor_base.py:113] # cuda blocks: 19911, # CPU blocks: 8192
INFO 07-23 09:02:37 [executor_base.py:118] Maximum concurrency for 131072 tokens per request: 2.43x
INFO 07-23 09:02:39 [model_runner.py:1513] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in 

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 07-23 09:03:13 [model_runner.py:1671] Graph capturing finished in 34 secs, took 0.12 GiB
INFO 07-23 09:03:13 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 38.08 seconds


Adding requests:   0%|          | 0/1 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

PassManager::run failed


## Basics

- The LLM class provides the primary Python interface for doing offline inference, which is interacting with a model without using a separate model inference server.

## Features

- Default generation config ( --generation-config auto)
- Quantization
  - AQLM

    Eg:

    - ISTA-DASLab/Mixtral-8x7b-AQLM-2Bit-1x16-hf
    - BlackSamorez/TinyLlama-1_1B-Chat-v1_0-AQLM-2Bit-1x16-hf

  - GGUF
  - CPU offload

In [ ]:
# GGUF
from huggingface_hub import hf_hub_download

repo_id = "bartowski/Phi-3-medium-4k-instruct-GGUF"
filename = "Phi-3-medium-4k-instruct-IQ2_M.gguf"

# model_path = hf_hub_download(repo_id=repo_id, filename=filename)

In [1]:
# Basic.py
from vllm import LLM, SamplingParams

prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

def main():
  llm = LLM(model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")   # meta-llama/Llama-3.2-1B
  outputs = llm.generate(prompts, sampling_params)
  print("\nGenerated Outputs:\n" + "-" * 60)
  for output in outputs:
          prompt = output.prompt
          generated_text = output.outputs[0].text
          print(f"Prompt:    {prompt!r}")
          print(f"Output:    {generated_text!r}")
          print("-" * 60)
if __name__ == "__main__":
    main()

INFO 07-23 09:17:10 [__init__.py:244] Automatically detected platform cuda.


config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

INFO 07-23 09:17:29 [config.py:841] This model supports multiple tasks: {'classify', 'reward', 'generate', 'embed'}. Defaulting to 'generate'.
WARNING 07-23 09:17:29 [config.py:3320] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 07-23 09:17:29 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 07-23 09:17:29 [config.py:1472] Using max model len 2048
WARNING 07-23 09:17:29 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 07-23 09:17:29 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='TinyLlama/TinyLlama-1.1B-Chat-v1.0', speculative_config=None, tokenizer='TinyLlama/TinyLlama-1.1B-Chat-v1.0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_forma

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

...

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 07-23 09:18:34 [model_runner.py:1671] Graph capturing finished in 33 secs, took 0.13 GiB
INFO 07-23 09:18:34 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 38.50 seconds


Adding requests:   0%|          | 0/4 [00:00<?, ?it/s]

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Generated Outputs:
------------------------------------------------------------
Prompt:    'Hello, my name is'
Output:    '\n20. How did the group meet and begin their journey together?\n'
------------------------------------------------------------
Prompt:    'The president of the United States is'
Output:    ' a member of the Cabinet.\nThe President of the United States is the'
------------------------------------------------------------
Prompt:    'The capital of France is'
Output:    ' Paris?'
------------------------------------------------------------
Prompt:    'The future of AI is'
Output:    ' cybersecurity\nThe future of AI is cybersecurity\nYesterday'
------------------------------------------------------------


In [ ]:
from vllm import LLM, EngineArgs
from vllm.utils import FlexibleArgumentParser

def create_parser():
  """Creats a parser"""
  parser = FlexibleArgumentParser()
  EngineArgs.add_cli_args(parser)

  parser.set_defaults(model="meta-llama/Llama-3.2-1B")
  sampling_group = parser.add_argument_group("Sampling Parameters")
  sampling_group.add_argument("--temperature", type=float, default=0.2)
  sampling_group.add_argument("--top_p", type=float, default=0.9)
  sampling_group.add_argument("--max_tokens", type=int, default=100)
  sampling_group.add_argument("--top-k", type=int, default=50)
  parser.add_argument("--chat-template-path", type=str)
  return parser

def main(args: dict):
  max_tokens = args.pop("max_tokens")
  temperature = args.pop("temperature")
  top_p = args.pop("top_p")
  top_k = args.pop("top_k")
  chat_template_path = args.pop("chat_template_path")

  llm = LLM(**args)
  # print(llm.get_default_sampling_params())

  sampling_params = llm.get_default_sampling_params()
  if max_tokens is not None:
        sampling_params.max_tokens = max_tokens
  if temperature is not None:
        sampling_params.temperature = temperature
  if top_p is not None:
        sampling_params.top_p = top_p
  if top_k is not None:
        sampling_params.top_k = top_k

  def print_outputs(outputs):
        print("\nGenerated Outputs:\n" + "-" * 80)
        for output in outputs:
            prompt = output.prompt
            generated_text = output.outputs[0].text
            print(f"Prompt: {prompt!r}\n")
            print(f"Generated text: {generated_text!r}")
            print("-" * 80)

  print("=" * 80)

  conversation = [
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
        {"role": "assistant", "content": "Hello! How can I assist you today?"},
        {"role": "user", "content": "Write an essay about the importance of higher education."},
    ]
  outputs = llm.chat(conversation, sampling_params, use_tqdm=False)
  print_outputs(outputs)

  conversations = [conversation for _ in range(10)]

  outputs = llm.chat(conversations, sampling_params, use_tqdm=True)
  print_outputs(outputs)

  if chat_template_path is not None:
        with open(chat_template_path) as f:
            chat_template = f.read()

        outputs = llm.chat(
            conversations,
            sampling_params,
            use_tqdm=False,
            chat_template=chat_template,
        )

if __name__ == "__main__":
    parser = create_parser()
    args: dict = vars(parser.parse_args())
    main(args)

## Batch LLM Inference

## Chat With Tools

## Context Extension

In [7]:
from vllm import LLM, SamplingParams

def create_llm():
    rope_theta = 1000000
    original_max_position_embeddings = 32768
    factor = 4.0

    hf_overrides = {
        "rope_theta": rope_theta,
        "rope_scaling": {
            "rope_type": "yarn",
            "factor": factor,
            "original_max_position_embeddings": original_max_position_embeddings,
        },
        "max_model_len": int(original_max_position_embeddings * factor),
    }

    llm = LLM(model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", hf_overrides=hf_overrides) # meta-llama/Llama-3.2-1B
    return llm

def run_llm_chat(llm):
    sampling_params = SamplingParams(
        temperature=0.8,
        top_p=0.95,
        max_tokens=128,
    )

    conversation = [
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
        {"role": "assistant", "content": "Hello! How can I assist you today?"},
    ]
    outputs = llm.chat(conversation, sampling_params, use_tqdm=False)
    return outputs

def print_outputs(outputs):
    print("\nGenerated Outputs:\n" + "-" * 80)
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}\n")
        print(f"Generated text: {generated_text!r}")
        print("-" * 80)


def main():
    llm = create_llm()
    outputs = run_llm_chat(llm)
    print_outputs(outputs)


if __name__ == "__main__":
    main()

INFO 07-23 09:51:09 [config.py:841] This model supports multiple tasks: {'embed', 'generate', 'reward', 'classify'}. Defaulting to 'generate'.
WARNING 07-23 09:51:09 [config.py:3320] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 07-23 09:51:09 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 07-23 09:51:09 [config.py:1472] Using max model len 131072
WARNING 07-23 09:51:09 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 07-23 09:51:09 [arg_utils.py:1531] Chunked prefill is enabled by default for models with max_model_len > 32K. Chunked prefill might not work with some features or models. If you encounter any issues, please disable by launching with --enable-chunked-prefill=False.
INFO 07-23 09:51:10 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 07-23 09:51:10 [llm_engine.py:230] Ini

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-23 09:51:21 [default_loader.py:272] Loading weights took 8.86 seconds
INFO 07-23 09:51:22 [model_runner.py:1203] Model loading took 2.0667 GiB and 9.484661 seconds
INFO 07-23 09:51:23 [worker.py:294] Memory profiling takes 0.96 seconds
INFO 07-23 09:51:23 [worker.py:294] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 07-23 09:51:23 [worker.py:294] model weights take 2.07GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 0.31GiB; the rest of the memory reserved for KV Cache is 10.85GiB.
INFO 07-23 09:51:24 [executor_base.py:113] # cuda blocks: 32311, # CPU blocks: 11915
INFO 07-23 09:51:24 [executor_base.py:118] Maximum concurrency for 131072 tokens per request: 3.94x
INFO 07-23 09:51:28 [model_runner.py:1513] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' i

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 07-23 09:52:02 [model_runner.py:1671] Graph capturing finished in 34 secs, took 0.13 GiB
INFO 07-23 09:52:02 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 39.94 seconds
INFO 07-23 09:52:04 [chat_utils.py:444] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


RuntimeError: PassManager::run failed

## Data Parallel

## Disaggregated Prefill V1

In [ ]:
# decoder example
# SPDX-License-Identifier: Apache-2.0
# SPDX-FileCopyrightText: Copyright contributors to the vLLM project

from vllm import LLM, SamplingParams
from vllm.config import KVTransferConfig


def read_prompts():
    """Read prompts from output.txt"""
    prompts = []
    try:
        with open("output.txt") as f:
            for line in f:
                prompts.append(line.strip())
        print(f"Loaded {len(prompts)} prompts from output.txt")
        return prompts
    except FileNotFoundError:
        print("Error: output.txt file not found")
        exit(-1)


def main():
    prompts = read_prompts()
    sampling_params = SamplingParams(temperature=0, top_p=0.95, max_tokens=10)

    llm = LLM(
        model="meta-llama/Llama-3.2-1B-Instruct",
        enforce_eager=True,
        gpu_memory_utilization=0.8,
        max_num_batched_tokens=64,
        max_num_seqs=16,
        kv_transfer_config=KVTransferConfig(
            kv_connector="SharedStorageConnector",
            kv_role="kv_both",
            kv_connector_extra_config={"shared_storage_path": "local_storage"},
        ),
    )  # , max_model_len=2048, max_num_batched_tokens=2048)

    # 1ST generation (prefill instance)
    outputs = llm.generate(prompts, sampling_params)

    print("-" * 30)
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}\nGenerated text: {generated_text!r}")
        print("-" * 30)


if __name__ == "__main__":
    main()

In [ ]:
# Prefill
# SPDX-License-Identifier: Apache-2.0
# SPDX-FileCopyrightText: Copyright contributors to the vLLM project

from vllm import LLM, SamplingParams
from vllm.config import KVTransferConfig


def read_prompts():
    context = "Hi " * 1000
    context2 = "Hey " * 500
    return [
        context + "Hello, my name is",
        context + "The capital of France is",
        context2 + "Your name is",
        context2 + "The capital of China is",
    ]


def main():
    prompts = read_prompts()

    sampling_params = SamplingParams(temperature=0, top_p=0.95, max_tokens=1)

    llm = LLM(
        model="meta-llama/Llama-3.2-1B-Instruct",
        enforce_eager=True,
        gpu_memory_utilization=0.8,
        kv_transfer_config=KVTransferConfig(
            kv_connector="SharedStorageConnector",
            kv_role="kv_both",
            kv_connector_extra_config={"shared_storage_path": "local_storage"},
        ),
    )  # , max_model_len=2048, max_num_batched_tokens=2048)

    # 1ST generation (prefill instance)
    outputs = llm.generate(
        prompts,
        sampling_params,
    )

    new_prompts = []
    print("-" * 30)
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        new_prompts.append(prompt + generated_text)
        print(f"Prompt: {prompt!r}\nGenerated text: {generated_text!r}")
        print("-" * 30)

    # Write new_prompts to output.txt
    with open("output.txt", "w") as f:
        for prompt in new_prompts:
            f.write(prompt + "\n")
    print(f"Saved {len(new_prompts)} prompts to output.txt")


if __name__ == "__main__":
    main()

## Embed Jina Embedding v3

In [ ]:
# SPDX-License-Identifier: Apache-2.0
# SPDX-FileCopyrightText: Copyright contributors to the vLLM project

from argparse import Namespace

from vllm import LLM, EngineArgs
from vllm.utils import FlexibleArgumentParser


def parse_args():
    parser = FlexibleArgumentParser()
    parser = EngineArgs.add_cli_args(parser)
    # Set example specific arguments
    parser.set_defaults(
        model="jinaai/jina-embeddings-v3", task="embed", trust_remote_code=True
    )
    return parser.parse_args()


def main(args: Namespace):
    # Sample prompts.
    prompts = [
        "Follow the white rabbit.",  # English
        "Sigue al conejo blanco.",  # Spanish
        "Suis le lapin blanc.",  # French
        "跟着白兔走。",  # Chinese
        "اتبع الأرنب الأبيض.",  # Arabic
        "Folge dem weißen Kaninchen.",  # German
    ]

    # Create an LLM.
    # You should pass task="embed" for embedding models
    model = LLM(**vars(args))

    # Generate embedding. The output is a list of EmbeddingRequestOutputs.
    # Only text matching task is supported for now. See #16120
    outputs = model.embed(prompts)

    # Print the outputs.
    print("\nGenerated Outputs:")
    print("Only text matching task is supported for now. See #16120")
    print("-" * 60)
    for prompt, output in zip(prompts, outputs):
        embeds = output.outputs.embedding
        embeds_trimmed = (
            (str(embeds[:16])[:-1] + ", ...]") if len(embeds) > 16 else embeds
        )
        print(
            f"Prompt: {prompt!r} \n"
            f"Embeddings for text matching: {embeds_trimmed} "
            f"(size={len(embeds)})"
        )
        print("-" * 60)


if __name__ == "__main__":
    args = parse_args()
    main(args)

## Embed Matryoshka Fy

In [ ]:
# SPDX-License-Identifier: Apache-2.0
# SPDX-FileCopyrightText: Copyright contributors to the vLLM project

from argparse import Namespace

from vllm import LLM, EngineArgs, PoolingParams
from vllm.utils import FlexibleArgumentParser


def parse_args():
    parser = FlexibleArgumentParser()
    parser = EngineArgs.add_cli_args(parser)
    # Set example specific arguments
    parser.set_defaults(
        model="jinaai/jina-embeddings-v3", task="embed", trust_remote_code=True
    )
    return parser.parse_args()


def main(args: Namespace):
    # Sample prompts.
    prompts = [
        "Follow the white rabbit.",  # English
        "Sigue al conejo blanco.",  # Spanish
        "Suis le lapin blanc.",  # French
        "跟着白兔走。",  # Chinese
        "اتبع الأرنب الأبيض.",  # Arabic
        "Folge dem weißen Kaninchen.",  # German
    ]

    # Create an LLM.
    # You should pass task="embed" for embedding models
    model = LLM(**vars(args))

    # Generate embedding. The output is a list of EmbeddingRequestOutputs.
    outputs = model.embed(prompts, pooling_params=PoolingParams(dimensions=32))

    # Print the outputs.
    print("\nGenerated Outputs:")
    print("-" * 60)
    for prompt, output in zip(prompts, outputs):
        embeds = output.outputs.embedding
        embeds_trimmed = (
            (str(embeds[:16])[:-1] + ", ...]") if len(embeds) > 16 else embeds
        )
        print(f"Prompt: {prompt!r} \nEmbeddings: {embeds_trimmed} (size={len(embeds)})")
        print("-" * 60)


if __name__ == "__main__":
    args = parse_args()
    main(args)

## Metrice

In [1]:
# SPDX-License-Identifier: Apache-2.0
# SPDX-FileCopyrightText: Copyright contributors to the vLLM project

from vllm import LLM, SamplingParams
from vllm.v1.metrics.reader import Counter, Gauge, Histogram, Vector

# Sample prompts.
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
# Create a sampling params object.
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)


def main():
    # Create an LLM.
    llm = LLM(model="meta-llama/Llama-3.2-1B-Instruct", disable_log_stats=False)

    # Generate texts from the prompts.
    outputs = llm.generate(prompts, sampling_params)

    # Print the outputs.
    print("-" * 50)
    for output in outputs:
        prompt = output.prompt
        generated_text = output.outputs[0].text
        print(f"Prompt: {prompt!r}\nGenerated text: {generated_text!r}")
        print("-" * 50)

    # Dump all metrics
    for metric in llm.get_metrics():
        if isinstance(metric, Gauge):
            print(f"{metric.name} (gauge) = {metric.value}")
        elif isinstance(metric, Counter):
            print(f"{metric.name} (counter) = {metric.value}")
        elif isinstance(metric, Vector):
            print(f"{metric.name} (vector) = {metric.values}")
        elif isinstance(metric, Histogram):
            print(f"{metric.name} (histogram)")
            print(f"    sum = {metric.sum}")
            print(f"    count = {metric.count}")
            for bucket_le, value in metric.buckets.items():
                print(f"    {bucket_le} = {value}")


if __name__ == "__main__":
    main()

INFO 07-23 10:19:16 [__init__.py:244] Automatically detected platform cuda.
INFO 07-23 10:19:35 [config.py:841] This model supports multiple tasks: {'generate', 'reward', 'classify', 'embed'}. Defaulting to 'generate'.
WARNING 07-23 10:19:35 [config.py:3320] Your device 'Tesla T4' (with compute capability 7.5) doesn't support torch.bfloat16. Falling back to torch.float16 for compatibility.
WARNING 07-23 10:19:35 [config.py:3371] Casting torch.bfloat16 to torch.float16.
INFO 07-23 10:19:35 [config.py:1472] Using max model len 131072
WARNING 07-23 10:19:35 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
WARNING 07-23 10:19:35 [arg_utils.py:1531] Chunked prefill is enabled by default for models with max_model_len > 32K. Chunked prefill might not work with some features or models. If you encounter any issues, please disable by launching with --enable-chunked-prefill=False.
INFO 07-23 10:19:35 [config.py:2285] Chunked prefill is enabled w

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 07-23 10:19:47 [default_loader.py:272] Loading weights took 8.51 seconds
INFO 07-23 10:19:48 [model_runner.py:1203] Model loading took 2.3185 GiB and 9.263028 seconds
INFO 07-23 10:19:49 [worker.py:294] Memory profiling takes 0.96 seconds
INFO 07-23 10:19:49 [worker.py:294] the current vLLM instance can use total_gpu_memory (14.74GiB) x gpu_memory_utilization (0.90) = 13.27GiB
INFO 07-23 10:19:49 [worker.py:294] model weights take 2.32GiB; non_torch_memory takes 0.05GiB; PyTorch activation peak memory takes 1.18GiB; the rest of the memory reserved for KV Cache is 9.72GiB.
INFO 07-23 10:19:49 [executor_base.py:113] # cuda blocks: 19911, # CPU blocks: 8192
INFO 07-23 10:19:49 [executor_base.py:118] Maximum concurrency for 131072 tokens per request: 2.43x
INFO 07-23 10:19:53 [model_runner.py:1513] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in 

Capturing CUDA graph shapes:   0%|          | 0/35 [00:00<?, ?it/s]

INFO 07-23 10:20:26 [model_runner.py:1671] Graph capturing finished in 33 secs, took 0.12 GiB
INFO 07-23 10:20:26 [llm_engine.py:428] init engine (profile, create kv cache, warmup model) took 38.53 seconds
INFO 07-23 10:20:28 [chat_utils.py:444] Detected the chat template content format to be 'string'. You can set `--chat-template-content-format` to override this.


TypeError: string indices must be integers, not 'str'

# RAG with Langchain

This script demonstrates a RAG implementation using LangChain, Milvus
and vLLM. RAG enhances LLM responses by retrieving relevant context
from a document collection.

Features:
- Web content loading and chunking
- Vector storage with Milvus
- Embedding generation with vLLM
- Question answering with context

Prerequisites:
1. Install dependencies:
        pip install -U vllm \
                 langchain_milvus langchain_openai \
                 langchain_community beautifulsoup4 \
                 langchain-text-splitters"

2. Start services:

    - Start embedding service (port 8000)
    
        vllm serve ssmits/Qwen2-7B-Instruct-embed-base

    - Start chat service (port 8001)

        vllm serve qwen/Qwen1.5-0.5B-Chat --port 8001

Usage:

    python retrieval_augmented_generation_with_langchain.py

Notes:

    - Ensure both vLLM services are running before executing
    - Default ports: 8000 (embedding), 8001 (chat)
    - First run may take time to download models

In [2]:
!pip install langchain_community langchain_ollama langchain_milvus  beautifulsoup4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.4/236.4 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 111.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.3/55.3 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 5.0 MB/s eta 0:00:00
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.73.1
    Uninstalling grpcio-1.73.1:
      Successfully uninstalled grpcio-1.73.1
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.69
    Uninstalling langchain-core-0.3.69:
      Successfully uninstalled langchain-core-0.3.69
ERROR: pip's dependency reso

In [3]:
import argparse
from argparse import ArgumentParser
from typing_extensions import Any

from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_milvus import Milvus
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [5]:
def load_and_split_documents(config: dict[str, Any]):
  """
    Load and split documents from web URL
  """
  try:
    loader = WebBaseLoader(config["url"])
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=config["chunk_size"],
      chunk_overlap=config["chunk_overlap"]
    )
    return text_splitter.split_documents(docs)
  except Exception as e:
        print(f"Error loading document from {config['url']}: {str(e)}")
        raise

In [7]:
def init_milvus_vectorstore(config: dict[str, Any], documents: list[Document]):
  """
    Initialize vector store with documents
  """
  return Milvus.from_documents(
        documents=documents,
        embedding=OllamaEmbeddings(
            model=config["embedding_model"],
            # openai_api_key=config["vllm_api_key"],
            # openai_api_base=config["vllm_embedding_endpoint"],
        ),
        connection_args={"uri": config["uri"]},
        drop_old=True,
    )

In [8]:
def init_llm(config: dict[str, Any]):
    """
    Initialize llm
    """
    return ChatOllama(model= config["chat_model"],)

In [9]:
def get_qa_prompt():
    """
    Get question answering prompt template
    """
    template = """You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question.
    If you don't know the answer, just say that you don't know.
    Use three sentences maximum and keep the answer concise.
    Question: {question}
    Context: {context}
    Answer:
    """
    return PromptTemplate.from_template(template)

In [10]:
def format_docs(docs: list[Document]):
    """
    Format documents for prompt
    """
    return "\n\n".join(doc.page_content for doc in docs)

In [11]:
def create_qa_chain(retriever: Any, llm: ChatOllama, prompt: PromptTemplate):
    """
    Set up question answering chain
    """
    return (
        {
            "context": retriever | format_docs,
            "question": RunnablePassthrough(),
        }
        | prompt
        | llm
        | StrOutputParser()
    )

In [12]:
def get_parser() -> argparse.ArgumentParser:
    """
    Parse command line arguments
    """
    parser = argparse.ArgumentParser(description="RAG with vLLM and langchain")

    # Add command line arguments
    parser.add_argument(
        "--vllm-api-key", default="EMPTY", help="API key for vLLM compatible services"
    )
    parser.add_argument(
        "--vllm-embedding-endpoint",
        default="http://localhost:8000/v1",
        help="Base URL for embedding service",
    )
    parser.add_argument(
        "--vllm-chat-endpoint",
        default="http://localhost:8001/v1",
        help="Base URL for chat service",
    )
    parser.add_argument("--uri", default="./milvus.db", help="URI for Milvus database")
    parser.add_argument(
        "--url",
        default=("https://docs.vllm.ai/en/latest/getting_started/quickstart.html"),
        help="URL of the document to process",
    )
    parser.add_argument(
        "--embedding-model",
        default="llama3.2:1b",
        help="Model name for embeddings",
    )
    parser.add_argument(
        "--chat-model", default="llama3.2:1b", help="Model name for chat"
    )
    parser.add_argument(
        "-i", "--interactive", action="store_true", help="Enable interactive Q&A mode"
    )
    parser.add_argument(
        "-k", "--top-k", type=int, default=3, help="Number of top results to retrieve"
    )
    parser.add_argument(
        "-c",
        "--chunk-size",
        type=int,
        default=1000,
        help="Chunk size for document splitting",
    )
    parser.add_argument(
        "-o",
        "--chunk-overlap",
        type=int,
        default=200,
        help="Chunk overlap for document splitting",
    )

    return parser

In [14]:
from argparse import Namespace

def init_config(args: Namespace):
    """
    Initialize configuration settings from command line arguments
    """

    return {
        "vllm_api_key": args.vllm_api_key,
        "vllm_embedding_endpoint": args.vllm_embedding_endpoint,
        "vllm_chat_endpoint": args.vllm_chat_endpoint,
        "uri": args.uri,
        "embedding_model": args.embedding_model,
        "chat_model": args.chat_model,
        "url": args.url,
        "chunk_size": args.chunk_size,
        "chunk_overlap": args.chunk_overlap,
        "top_k": args.top_k,
    }


In [15]:
def main():
    # Parse command line arguments
    args = get_parser().parse_args()

    # Initialize configuration
    config = init_config(args)

    # Load and split documents
    documents = load_and_split_documents(config)

    # Initialize vector store and retriever
    vectorstore = init_milvus_vectorstore(config, documents)
    retriever = vectorstore.as_retriever(search_kwargs={"k": config["top_k"]})

    # Initialize llm and prompt
    llm = init_llm(config)
    prompt = get_qa_prompt()

    # Set up QA chain
    qa_chain = create_qa_chain(retriever, llm, prompt)

    # Interactive mode
    if args.interactive:
        print("\nWelcome to Interactive Q&A System!")
        print("Enter 'q' or 'quit' to exit.")

        while True:
            question = input("\nPlease enter your question: ")
            if question.lower() in ["q", "quit"]:
                print("\nThank you for using! Goodbye!")
                break

            output = qa_chain.invoke(question)
            print(output)
    else:
        # Default single question mode
        question = "How to install vLLM?"
        output = qa_chain.invoke(question)
        print("-" * 50)
        print(output)
        print("-" * 50)

In [16]:
if __name__ == "__main__":
  question = input("\nPlease enter your question: ")



usage: colab_kernel_launcher.py [-h] [--vllm-api-key VLLM_API_KEY]
                                [--vllm-embedding-endpoint VLLM_EMBEDDING_ENDPOINT]
                                [--vllm-chat-endpoint VLLM_CHAT_ENDPOINT]
                                [--uri URI] [--url URL]
                                [--embedding-model EMBEDDING_MODEL]
                                [--chat-model CHAT_MODEL] [-i] [-k TOP_K]
                                [-c CHUNK_SIZE] [-o CHUNK_OVERLAP]
colab_kernel_launcher.py: error: unrecognized arguments: -f /root/.local/share/jupyter/runtime/kernel-d09c0b94-3fba-4cac-9e19-132a51a2a822.json


SystemExit: 2

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
